In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score 
from pandas.api.types import is_numeric_dtype
import warnings
from sklearn import tree
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/network-intrusion-detection/Train_data.csv")

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
def check_constant_columns(train):
    constant_columns = [col for col in train.columns if train[col].nunique() == 1]
    return constant_columns

# Example usage
constant_cols = check_constant_columns(train)
if constant_cols:
    print(f"Columns with the same value across all rows: {constant_cols}")
else:
    print("No columns have the same value across all rows.")


In [ ]:
#Dropping a column because every value is 0 
train.drop(['num_outbound_cmds'], axis=1, inplace=True)
train.drop(['is_host_login'], axis=1, inplace=True)

In [ ]:
# Splitting the Training Dataset into X_train (features) and y_train (label)
X = train.drop(["class"], axis=1)
y= train["class"]


In [ ]:
#Apply LabelEncoding to all the categorical columns (those with data type 'object')

from sklearn.preprocessing import LabelEncoder
 
def LabelEncoding(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

LabelEncoding(train)

In [ ]:
# Splitting the Training Dataset into X_train (features) and y_train (label)
X = train.drop(["class"], axis=1)
y= train["class"]


In [ ]:
print(f"{X.shape}")
print(f"{y.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=42)

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 
np.random.seed(42)

# #Instantiate Random Forest Classifier
# clf = RandomForestClassifier(n_estimators=5)

# #Fitting the model to the data 
# clf.fit(X_train, y_train.values.ravel())

# Choose a range of n_estimators to evaluate
#n_estimators_range = range(1, 101, 10)  # Example: 1, 11, ..., 91
#Starting estimators from 10
n_estimators_range = range(10, 110, 10)
# Arrays to store accuracies
train_accuracies = []
validation_accuracies = []

# Loop through each number of trees
for n_estimators in n_estimators_range:
    clf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    clf.fit(X_train, y_train)
    
    # Training accuracy
    y_train_preds = clf.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_preds)
    train_accuracies.append(train_accuracy)
    
    # Validation accuracy
    y_test_preds = clf.predict(X_test)
    validation_accuracy = accuracy_score(y_test, y_test_preds)
    validation_accuracies.append(validation_accuracy)

# Convert accuracies to "loss" (1 - accuracy) to plot
train_losses = [1 - acc for acc in train_accuracies]
validation_losses = [1 - acc for acc in validation_accuracies]

In [ ]:
# # Calculate the training loss (1 - accuracy)
# train_pred = clf.predict(X_train)
# train_accuracy = accuracy_score(y_train, train_pred)
# train_loss = 1 - accuracy_score(y_train, train_pred)
# print(f" train loss: {train_loss * 100:.2f}%")

In [ ]:
# # Calculate the validation loss (1 - accuracy)
# print(f"validation loss: {val_loss * 100:.2f}%")

In [ ]:
# Plot training and validation losses
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_range, train_losses, label='Training Loss', marker='o')
plt.plot(n_estimators_range, validation_losses, label='Validation Loss', marker='o')
plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Loss (1 - Accuracy)')
plt.title('Training and Validation Loss vs. Number of Trees')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(y_test, y_test_preds)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Plotting the training and validation accuracies
plt.figure(figsize=(12, 6))
plt.plot(n_estimators_range, train_accuracies, marker='o', color='green', label='Training Accuracy')
plt.plot(n_estimators_range, validation_accuracies, marker='o', label='Validation Accuracy')
plt.xlabel("Number of Trees (n_estimators)")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy vs. Number of Trees")
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Make some prediction
y_preds= clf.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y_test, y_preds))